# Executing Gradient-free Evasion Attacks

It is not always possible to instantiate gradient-based attacks, since the target model could be non-differentiable or simply not available to the attacker.

In these cases, the security of AI-based detectors can be computed with *gradient-free* techniques, by estimating the directions to follow in the search space through heuristics.
**All models** can be attacked with these strategies, so their application is fundamental in this field.

We start by loading models, and we later instantiate attacks.

In [1]:
%%capture --no-stderr
try:
    import maltorch
except ImportError:
   %pip install git+https://github.com/zangobot/maltorch

In [2]:
from pathlib import Path
from maltorch.data.loader import load_from_folder, create_labels
from maltorch.data_processing.grayscale_preprocessing import GrayscalePreprocessing
from maltorch.zoo.avaststyleconv import AvastStyleConv
from maltorch.zoo.bbdnn import BBDnn
from maltorch.zoo.malconv import MalConv
from maltorch.zoo.resnet18 import ResNet18
from maltorch.zoo.ember_gbdt import EmberGBDT
from secmlt.metrics.classification import Accuracy
from torch.utils.data import DataLoader, TensorDataset

exe_folder = Path("insert_here_path")
device = "cpu"
networks = {
    'BBDnn': BBDnn.create_model(device=device),
    'Malconv': MalConv.create_model(device=device),
    'EMBER GBDT': EmberGBDT.create_model(),
    'AvastStyleConv': AvastStyleConv.create_model(device=device),
    'Grayscale ResNet18': ResNet18.create_model(
        preprocessing=GrayscalePreprocessing(),
        device=device),
}

X = load_from_folder(exe_folder, "exe", device=device)
y = create_labels(X, 1)
data_loader = DataLoader(TensorDataset(X, y), batch_size=3)

RuntimeError: received an empty list of sequences

All attacks support gradient-free optimziation thanks to `nevergrad`.
This library provides scalable and fast gradient-free optimization algorithms.
To switch to this mode, we need to select the right backend for the attack.

In [ ]:
from maltorch.adv.evasion.content_shift import ContentShift
from maltorch.adv.evasion.base_optim_attack_creator import OptimizerBackends

# The instantiation of the attack is still the same, 
# but with the difference that the backend now requires Nevergrad.

grad_attack = ContentShift(
    query_budget=10,
    device=device,
    perturbation_size=2048,
    step_size=256,
    backend=OptimizerBackends.NG,
)

for k in networks:
    print(k)
    model = networks[k]
    adv_dl = grad_attack(model, data_loader)
    print("- - - Pre-attack accuracy: ", Accuracy()(model, data_loader))
    print("- - - Accuracy: ", Accuracy()(model, adv_dl))

That's it! You can swap attacks with ease, by just instantiating them accoredingly.